<a href="https://colab.research.google.com/github/zbooster/Landmark-Image-Classification/blob/main/Hyperparameter_Sweeps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 랜드마크 이미지 분류하기

In [ ]:
!nvidia-smi

Mon Aug  8 10:18:01 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Introduction

## Weights & Biases 설정

### Package install

In [ ]:
!pip install wandb -Uq

     |████████████████████████████████| 1.8 MB 17.0 MB/s 
     |████████████████████████████████| 157 kB 65.3 MB/s 
     |████████████████████████████████| 181 kB 70.6 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 157 kB 45.8 MB/s 
     |████████████████████████████████| 156 kB 58.0 MB/s 


### Login

In [ ]:
import wandb

wandb.login()

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

## Config

### Sweep config

모든 파라미터를 다 테스트하기 위해 grid

In [ ]:
sweep_config = {
    'method': 'grid'
}

In [ ]:
metric = {
    'name': 'Validation Accuracy',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

In [ ]:
parameters_dict = {
    'batch_size': {
        'value': 64 },
    'resize_pixel': {
        'value': 256},
    'crop_scale': {
        'value': 0.9},
    'learning_rate': {
        'value': 0.001},
    'grad_false_idx': {
        'value': 6 },
    'step_size': {
        'value': 6 },
    'gamma': {
        'value': 0.1},
    'epochs': {
        'value': 25},
    'class_size': {
        'value': 84},
    }

sweep_config['parameters'] = parameters_dict

In [ ]:
import pprint

pprint.pprint(sweep_config)

{'method': 'grid',
 'metric': {'goal': 'minimize', 'name': 'Validation Accuracy'},
 'parameters': {'batch_size': {'value': 64},
                'class_size': {'value': 84},
                'crop_scale': {'value': 0.9},
                'epochs': {'value': 25},
                'gamma': {'value': 0.1},
                'grad_false_idx': {'value': 6},
                'learning_rate': {'value': 0.001},
                'resize_pixel': {'value': 256},
                'step_size': {'value': 6}}}


### Tuning config

## Initialize the Sweep

## Dataset and DataLoader
수집된 원본데이터가 Colab에서 사용하기 너무 크기때문에 Local Computer에서 OpenCV를 이용하여 Resize를 수행한 결과를 파일로 가져와서 사용한다

In [ ]:
!gdown 1_yGDePPvjlsTe4qi3X6LqY2DPEB97Itq
!unzip -qq '/content/resizeds_0.1x.zip' -d './datasets'

Downloading...
From: https://drive.google.com/uc?id=1_yGDePPvjlsTe4qi3X6LqY2DPEB97Itq
To: /content/resizeds_0.1x.zip
100% 837M/837M [00:11<00:00, 71.6MB/s]


In [ ]:
sweep_id = wandb.sweep(sweep_config, project="Hyperparameter-Sweeps")

Create sweep with ID: cpxmwkmy
Sweep URL: https://wandb.ai/zbooster/Hyperparameter-Sweeps/sweeps/cpxmwkmy


### 이미지 픽셀의 평균, 표준편차를 계산하여 정규화 하기

In [ ]:
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

def get_mean_and_std(dataloader):
    channels_sum, channels_squared_sum, num_batches = 0, 0, 0
    for data, _ in dataloader:
        # Mean over batch, height and width, but not over the channels
        channels_sum += torch.mean(data, dim=[0,2,3])
        channels_squared_sum += torch.mean(data**2, dim=[0,2,3])
        num_batches += 1
    
    mean = channels_sum / num_batches

    # std = sqrt(E[X^2] - (E[X])^2)
    std = (channels_squared_sum / num_batches - mean ** 2) ** 0.5

    return mean, std

def build_nomalize_loader(set_type, resize_pixel, batch_size):
    transform = transforms.Compose([
        transforms.Resize([resize_pixel, resize_pixel]),
        transforms.ToTensor()
        ])

    dataset = ImageFolder(root='/content/datasets/resizeds/' + set_type, transform=transform)
    loader = DataLoader(dataset=dataset, batch_size=batch_size)

    return loader

# get_mean_and_std(build_nomalize_loader('Training', 256, 256))

In [ ]:
import os

def build_dataset(batch_size, set_type, resize_pixel, crop_scale): #, n_mean, n_std):
  
    compose_list = []
    compose_list.append(transforms.Resize([resize_pixel, resize_pixel]))
    # if set_type == 'Training':
    #   compose_list.append(transforms.RandomHorizontalFlip())
    #   compose_list.append(transforms.RandomVerticalFlip())
    # compose_list.append(transforms.RandomCrop(int(resize_pixel * crop_scale)))
    compose_list.append(transforms.ToTensor())
    # compose_list.append(transforms.Normalize(n_mean, n_std))
    
    transform = transforms.Compose(compose_list)
  
    data_dir = '/content/datasets/resizeds'
    dataset = ImageFolder(root=os.path.join(data_dir, set_type), transform=transform)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=2)
  
    return loader, len(dataset)

In [ ]:
import torch
from torchvision.models import resnet50, ResNet50_Weights
import torch.nn as nn
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def build_network(class_size):
    network = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
    num_ftrs = network.fc.in_features
    network.fc = nn.Linear(num_ftrs, class_size)
    return network.to(device)

In [ ]:
def build_optimizer(network, learning_rate, grad_false_idx):
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, network.parameters()), 
                           lr=learning_rate)
    for idx, child in enumerate(network.children()):
      if idx < grad_false_idx:
        for param in child.parameters():
          param.requires_grad = False
    return optimizer

In [ ]:
from torch.optim import lr_scheduler

def build_scheduler(optimizer, step_size, gamma):
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)
    return exp_lr_scheduler

In [ ]:
critetion = nn.CrossEntropyLoss()

def train_epoch(network, loader, datasize, optimizer, scheduler):
    network.train()

    running_loss = 0.0
    running_corrects = 0

    for _, (data, target) in enumerate(loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()

        with torch.set_grad_enabled(True):
          # ➡ Forward pass
          outputs = network(data)
          _, preds = torch.max(outputs, 1)
          loss = critetion(outputs, target)

          # ⬅ Backward pass + weight update
          loss.backward()
          optimizer.step()

        running_loss += loss.item() * data.size(0)
        running_corrects += torch.sum(preds == target.data)

    scheduler.step()

    epoch_loss = running_loss/datasize
    epoch_acc = running_corrects.double()/datasize

    return epoch_loss, epoch_acc

In [ ]:
def validation_epoch(network, loader, datasize, optimizer):
    network.eval()
    
    running_loss = 0.0
    running_corrects = 0

    for _, (data, target) in enumerate(loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()

        # ➡ Forward pass
        outputs = network(data)
        _, preds = torch.max(outputs, 1)
        loss = critetion(outputs, target)

        running_loss += loss.item() * data.size(0)
        running_corrects += torch.sum(preds == target.data)

    epoch_loss = running_loss/datasize
    epoch_acc = running_corrects.double()/datasize

    return epoch_loss, epoch_acc

In [ ]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
import time
from torchvision import datasets, transforms

def train(config=None):
    # Initialize a new wandb run
    with wandb.init(config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config

        # t_mean, t_std = get_mean_and_std(build_nomalize_loader('Training', config.resize_pixel, config.batch_size))
        t_loader, t_datasize = build_dataset(config.batch_size, 'Training', 
                                             config.resize_pixel, config.crop_scale
                                            #  ,t_mean, t_std
                                             )
        
        # v_mean, v_std = get_mean_and_std(build_nomalize_loader('Validation', config.resize_pixel, config.batch_size))
        v_loader, v_datasize = build_dataset(config.batch_size, 'Validation', 
                                             config.resize_pixel, config.crop_scale
                                            #  ,v_mean, v_std
                                             )
        
        network = build_network(config.class_size)
        optimizer = build_optimizer(network, config.learning_rate, config.grad_false_idx)
        scheduler = build_scheduler(optimizer, config.step_size, config.gamma)

        for epoch in range(config.epochs):
            since = time.time()
            
            epoch_loss, epoch_acc = train_epoch(network, t_loader, t_datasize, optimizer, scheduler)
            wandb.log({"Train Loss": epoch_loss, "Train Accuracy": epoch_acc * 100, "Epoch": epoch})

            epoch_loss, epoch_acc = validation_epoch(network, v_loader, v_datasize, optimizer)
            wandb.log({"Validation Loss": epoch_loss, "Validation Accuracy": epoch_acc * 100, "Epoch": epoch})

            time_elapsed = time.time() - since
            wandb.log({"Time Elapsed(s)": time_elapsed, "Epoch": epoch})


            

In [ ]:
wandb.agent(sweep_id, train)

wandb: Agent Starting Run: kw8nxzdx with config:
wandb: 	batch_size: 64
wandb: 	class_size: 84
wandb: 	crop_scale: 0.9
wandb: 	epochs: 25
wandb: 	gamma: 0.1
wandb: 	grad_false_idx: 6
wandb: 	learning_rate: 0.001
wandb: 	resize_pixel: 256
wandb: 	step_size: 6


Epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
Time Elapsed(s),█▁▂▂▂▂▂▂▂▂▁▂▁▂▂▂▁▂▂▁▂▁▂▁▂
Train Accuracy,▁█▇▇█████████████████████
Train Loss,█▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Validation Accuracy,▁▇▂▃▇████████████████████
Validation Loss,█▂▇▆▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Epoch,24
Time Elapsed(s),69.17534
Train Accuracy,100.0
Train Loss,9e-05
Validation Accuracy,99.80053


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
